In [1]:
import os
import pandas as pd
import json
import traceback
import PyPDF2
import requests

In [2]:
import huggingface_hub
print(huggingface_hub.__version__)

0.34.4


In [3]:
# from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import LLMResult
from langchain_community.llms import Ollama

In [4]:
print(os.path.isfile(r"C:\Users\uniba\McqGen\.env"))
print(os.getcwd())

True
c:\Users\uniba\McqGen\experiment


In [5]:
from dotenv import load_dotenv

load_dotenv(r"C:\Users\uniba\McqGen\.env")
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [6]:
llm = Ollama(model="tinyllama")

C:\Users\uniba\AppData\Local\Temp\ipykernel_9472\771404339.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="tinyllama")


In [7]:
# from langchain_core.language_models import BaseLLM
# from langchain_core.outputs import LLMResult, Generation
# from pydantic import Field

# class SimpleHuggingFace(BaseLLM):
#     api_token: str = Field()  # ← Add this instead of __init__
    
#     def _generate(self, prompts, stop=None, run_manager=None):
#         responses = []
#         for prompt in prompts:
#             response = requests.post( 
#                 "https://api-inference.huggingface.co/models/google/flan-t5-large",
#                 headers={"Authorization": f"Bearer {self.api_token}"},
#                 json={"inputs": prompt, "parameters": {"max_length": 800, "temperature": 0.7}}
#             )
#             text = response.json()[0]['generated_text']
#             responses.append(Generation(text=text))
#         return LLMResult(generations=[responses])
    
#     @property
#     def _llm_type(self) -> str:
#         return "huggingface"

In [8]:
# class SimpleHuggingFace:
#     def __init__(self, api_token):
#         self.api_token = api_token
    
#     def __call__(self, prompt):
#         response = requests.post(
#             "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1",
#             headers={"Authorization": f"Bearer {self.api_token}"},
#             json={"inputs": prompt, "parameters": {"max_length": 800, "temperature": 0.7}}
#         )
        
#         # ADD THESE 2 LINES FOR ERROR HANDLING:
#         if response.status_code != 200:
#             return f"API Error: {response.text}"
        
#         return response.json()[0]['generated_text']

In [9]:
# llm=HuggingFaceHub(
#     repo_id = "mistralai/Mistral-7B-v0.1",
#     model_kwargs={"temperature": 0.7, "max_length": 800},
#     huggingfacehub_api_token=KEY
# )

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
Template="""
You are a **master quiz architect** hired to design a dazzling set of {number} multiple-choice questions.

Theme: **{subject}**
Difficulty Target: **{levels}**

Source material for inspiration:
---------------------------------
{text}
---------------------------------

 **Your Mission**
Craft questions that:
1. Spark curiosity and test true understanding—no simple fact-recall.
2. Each question must have exactly 4 distinct options (A–D), with one correct answer.
3. Mix question types: conceptual, application, or a tiny real-world twist when possible.

🖼 **Formatting Canvas**
Respond *only* in valid JSON that matches this exact skeleton:
{response_json}

 Tips for Style
- Make the wrong answers *plausible* to challenge learners.
- Keep language concise and friendly, like a great teacher setting an exciting quiz.
- Vary question openings: “Which statement…”, “What would happen if…”, “Identify the…”, etc.

Deliver a JSON array that a program can parse immediately—no extra commentary.
"""""""""

In [12]:
quiz_prompt = PromptTemplate(
    input_variables=["text", "subject", "levels", "number", "response_json"],
    template=Template
)

In [13]:

quiz_chain=LLMChain(llm=llm, prompt=quiz_prompt, output_key="quiz", verbose=True)

C:\Users\uniba\AppData\Local\Temp\ipykernel_9472\3800351965.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_prompt, output_key="quiz", verbose=True)


In [14]:
Template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_review_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=Template2
)

In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_review_prompt, verbose=True, output_key="review")

In [17]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "subject", "levels", "number", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [18]:
file_path=r"C:\Users\uniba\McqGen\data.txt"

In [19]:
with open (file_path, 'r') as file:
    TEXT = file.read()

In [20]:
json.dumps(RESPONSE_JSON) # Serialize the dictionary into json formatted string

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5
SUBJECT="chemistry"
LEVEL="simple"

In [22]:
response=generate_evaluate_chain(
    {
        "text": TEXT,
        "subject": SUBJECT,
        "levels": LEVEL,
        "number": NUMBER,
        "response_json": json.dumps(RESPONSE_JSON)
    }
    )

C:\Users\uniba\AppData\Local\Temp\ipykernel_9472\1440702647.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a **master quiz architect** hired to design a dazzling set of 5 multiple-choice questions.

Theme: **chemistry**
Difficulty Target: **simple**

Source material for inspiration:
---------------------------------
Chemistry is the scientific study of the properties and behavior of matter.[1][2] It is a physical science within the natural sciences that studies the chemical elements that make up matter and compounds made of atoms, molecules and ions: their composition, structure, properties, behavior and the changes they undergo during reactions with other substances.[3][4][5][6] Chemistry also addresses the nature of chemical bonds in chemical compounds.

In the scope of its subject, chemistry occupies an intermediate position between physics and biology.[7] It is sometimes called the central science because it provides a foundation for understanding both basic and applied scienti

In [24]:
quiz_str = response.get("quiz")

# ADD THIS ERROR HANDLING:
try:
    quiz_dict = json.loads(quiz_str)
except json.JSONDecodeError:
    print("AI didn't return valid JSON. Raw output:")
    print(quiz_str)
    quiz_dict = {}  # Empty dictionary to prevent crash

quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

df=pd.DataFrame(quiz_table_data)

AI didn't return valid JSON. Raw output:
To deliver a well-structured and valid JSON array for your quiz, here's an example with the correct skeleton:

```json
[
    {
        "mcq": "multiple choice question",
        "options": [
            {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"},
            {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"},
            {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}
        ],
        "correct": "correct answer"
    },
    {
        "mcq": "multiple choice question",
        "options": [
            {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"},
            {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"},
            {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}
        ],
        "correct": "correct answer"
    },
    {
        "mcq": "multip